# Part 1

In [1]:
import functools
import operator

class CPU:
    def __init__(self, registers):
        self._registers = list(registers)

    @property
    def registers(self):
        return tuple(self._registers)
    
    def _do_op_imm(self, op, a, b, c):
        self._registers[c] = op(self._registers[a], b)
        
    def _do_op_reg(self, op, a, b, c):
        self._registers[c] = op(self._registers[a], self._registers[b])

    addi = functools.partialmethod(_do_op_imm, operator.add)
    addr = functools.partialmethod(_do_op_reg, operator.add)
    muli = functools.partialmethod(_do_op_imm, operator.mul)
    mulr = functools.partialmethod(_do_op_reg, operator.mul)
    bani = functools.partialmethod(_do_op_imm, operator.and_)
    banr = functools.partialmethod(_do_op_reg, operator.and_)
    bori = functools.partialmethod(_do_op_imm, operator.or_)
    borr = functools.partialmethod(_do_op_reg, operator.or_)

    def _do_cmp_reg_imm(self, cmp, a, b, c):
        self._registers[c] = int(cmp(self._registers[a], b))
        
    def _do_cmp_imm_reg(self, cmp, a, b, c):
        self._registers[c] = int(cmp(a, self._registers[b]))

    def _do_cmp_reg_reg(self, cmp, a, b, c):
        self._registers[c] = int(cmp(self._registers[a], self._registers[b]))

    gtir = functools.partialmethod(_do_cmp_imm_reg, operator.gt)
    gtri = functools.partialmethod(_do_cmp_reg_imm, operator.gt)
    gtrr = functools.partialmethod(_do_cmp_reg_reg, operator.gt)
    eqir = functools.partialmethod(_do_cmp_imm_reg, operator.eq)
    eqri = functools.partialmethod(_do_cmp_reg_imm, operator.eq)
    eqrr = functools.partialmethod(_do_cmp_reg_reg, operator.eq)

    def seti(self, a, b, c):    
        self._registers[c] = a
        
    def setr(self, a, b, c):
        self._registers[c] = self._registers[a]


In [2]:
cpu = CPU((1,2,3,4))
cpu.addi(1, 5, 2)
assert cpu.registers == (1, 2, 7, 4)

In [3]:
cpu = CPU((1,1,1,1))
cpu.addr(0, 1, 3)
assert cpu.registers == (1, 1, 1, 2)

In [4]:
cpu = CPU((1,0,0,0))
cpu.bori(0, 2, 1)
assert cpu.registers == (1, 3, 0, 0)

In [5]:
cpu = CPU((7,3,0,0))
cpu.banr(0, 1, 2)
assert cpu.registers == (7, 3, 3, 0)

In [6]:
cpu = CPU((2,1,0,0))
cpu.gtrr(0, 1, 2)
assert cpu.registers == (2, 1, 1, 0)

In [7]:
cpu = CPU((1,2,1,0))
cpu.gtrr(0, 1, 2)
assert cpu.registers == (1, 2, 0, 0)

In [8]:
cpu = CPU((2,1,9,9))
cpu.eqir(1, 1, 2)
assert cpu.registers == (2, 1, 1, 9)

In [9]:
cpu = CPU((2,1,9,9))
cpu.eqri(0, 1, 2)
assert cpu.registers == (2, 1, 0, 9)

In [10]:
from collections import namedtuple

ExampleCommand = namedtuple('ExampleCommand', 'before instruction after')

In [11]:
def test_command(cmd):
    ''' Test a command against all known instructions and count how many are compatible. '''
    instructions = ['addi','addr','muli','mulr','bani','banr','bori','borr',
                    'gtir','gtri','gtrr','eqir','eqri','eqrr','seti','setr']
    count = 0
    for instruction in instructions:
        cpu = CPU(cmd.before)
        getattr(cpu, instruction)(*cmd.instruction[1:4])
        if cpu.registers == cmd.after:
            count += 1
    return count

In [12]:
test = ExampleCommand((3,2,1,1), (9,2,1,2), (3,2,2,1))
print(test)
test_command(test)

ExampleCommand(before=(3, 2, 1, 1), instruction=(9, 2, 1, 2), after=(3, 2, 2, 1))


3

In [13]:
import re

register_re = re.compile(r'(Before|After):\s+\[(\d), (\d), (\d), (\d)]')

def parse_input(text_):
    ''' Parse input file up to the two blank lines. Return list of ExampleCommand. 
    Updated for part 2: return example commands and list of (op,a,b,c).'''
    # Parse data for part 1.
    examples = list()
    lines = iter(text_.split('\n'))
    while True:
        line = next(lines)
        if not line.startswith('Before:'):
            break
        before = tuple([int(i) for i in register_re.match(line).group(2,3,4,5)])
        instruction = tuple([int(i) for i in next(lines).split()])
        after = tuple([int(i) for i in register_re.match(next(lines)).group(2,3,4,5)])
        examples.append(ExampleCommand(before, instruction, after))
        next(lines)
    next(lines)

    # By the time you get here, the next line will be the data for part 2.
    program = list()
    while True:
        try:
            line = next(lines)
        except StopIteration:
            break
        program.append(tuple([int(i) for i in line.split()]))
    return examples, program

In [14]:
with open('input.txt') as input_:
    examples, program = parse_input(input_.read())
print(len(examples))
print(examples[0])
print(len(program))
print(program[0])

761
ExampleCommand(before=(0, 2, 2, 2), instruction=(4, 2, 3, 2), after=(0, 2, 5, 2))
969
(1, 0, 2, 3)


In [17]:
def count_examples(examples):
    ''' Count how many commands are compatible with 3 or more opcodes. '''
    count = 0
    for example in examples:
        if test_command(example) >= 3:
            count += 1
    return count

In [18]:
count_commands(examples)

521

# Part 2

In [43]:
def test_command2(cmd, instructions):
    ''' Similar to test_command() but returns set of instructions that are compatible. '''
    compatible = set()
    for instruction in instructions:
        cpu = CPU(cmd.before)
        getattr(cpu, instruction)(*cmd.instruction[1:4])
        if cpu.registers == cmd.after:
            compatible.add(instruction)
    return compatible

In [49]:
from collections import defaultdict

def identify_opcodes(examples):
    ''' Identify all examples that can only be 1 opcode. Assign names to those
    numbers, then remove those from the set of candidates and try again. Returns
    a dict that maps opcode numbers to names. '''
    opcodes = dict()
    instructions = {'addi','addr','muli','mulr','bani','banr','bori','borr',
                    'gtir','gtri','gtrr','eqir','eqri','eqrr','seti','setr'}
    
    while instructions:
        compatibility = defaultdict(set)
        
        # For each opcode, check how many of the remaining instructions are 
        # compatible with it
        for example in examples:
            opcode = example.instruction[0]
            compatibility[opcode].update(test_command2(example, instructions))

        # For any opcode with only 1 compatible instruction, we record the
        # mapping and remove that instruction from the set of candidates.
        for opcode, compatible in compatibility.items():
            if len(compatible) == 1:
                instruction = compatible.pop()
                opcodes[opcode] = instruction
                instructions.remove(instruction)

        print('I know {} opcodes and have {} instructions remaining.'.format(
            len(opcodes), len(instructions)))

    return opcodes

In [50]:
opcodes = identify_opcodes(examples)

I know 1 opcodes and have 15 instructions remaining.
I know 2 opcodes and have 14 instructions remaining.
I know 3 opcodes and have 13 instructions remaining.
I know 4 opcodes and have 12 instructions remaining.
I know 5 opcodes and have 11 instructions remaining.
I know 6 opcodes and have 10 instructions remaining.
I know 7 opcodes and have 9 instructions remaining.
I know 8 opcodes and have 8 instructions remaining.
I know 9 opcodes and have 7 instructions remaining.
I know 10 opcodes and have 6 instructions remaining.
I know 11 opcodes and have 5 instructions remaining.
I know 12 opcodes and have 4 instructions remaining.
I know 13 opcodes and have 3 instructions remaining.
I know 14 opcodes and have 2 instructions remaining.
I know 15 opcodes and have 1 instructions remaining.
I know 16 opcodes and have 0 instructions remaining.


In [48]:
opcodes

{4: 'addi',
 15: 'mulr',
 8: 'muli',
 14: 'addr',
 11: 'borr',
 9: 'bori',
 1: 'seti',
 7: 'gtri',
 3: 'eqrr',
 2: 'eqri',
 0: 'eqir',
 6: 'gtrr',
 12: 'gtir',
 5: 'setr',
 13: 'banr',
 10: 'bani'}

In [51]:
def execute_program(program, opcodes):
    ''' Execute the program represented as a list of (opcode,a,b,c) tuples using the
    given lookup table of opcodes. '''
    cpu = CPU((0,0,0,0))
    for instruction in program:
        op = getattr(cpu, opcodes[instruction[0]])
        args = instruction[1:4]
        op(*args)
    print(cpu)

In [52]:
execute_program(program, opcodes)

IndexError: tuple index out of range